<div>
     <div>
        <img src="./report/isel_logo.png" width="400" height="400" align="left">
    </div>
    <div>
        <h2>Área Departamental de Engenharia de Eletrónica e Telecomunicações e de Computadores</h2>
        <p>Trabalho prático 2</p>
        <p>Autor:	44598	André L. A. Q. de Oliveira</p>
        <p>Unidade Curricular Compressão de Sinais Multimédia</p>
        <p>Professor: André Lourenço</p>
        <p>09 - Maio - 2021</p>
    </div>
</div>

### <a id="index"></a>

# Index
- [Codificação de Huffman](#codificacao_huffman)
- [I/O Utilities](#io_utilities)
    - [pad_bits](#pad_bits)
    - [to_binary_list](#to_binary_list)
    - [InputBitReader](#input_bit_reader)
- [Tabela de Huffman](#tabela_huffman)
    - [get_symbol_frequency](#get_symbol_frequency)
    - [huff_node](#huff_node)
    - [create_huff_tree](#create_huff_tree)
    - [huff_tree_encode](#huff_tree_encode)
    - [huff_tree_decode](#huff_tree_decode)
    - [huff_tree2huff_dictionary](#huff_tree2huff_dictionary)
    - [gen_huff_table](#gen_huff_table)
- [Codificador de Huffman](#codificador_huffman)
    - [encode_huff](#encode_huff)
- [Descodificador de Huffman](#descodificador_huffman)
    - [decode_huff](#decode_huff)
- [Escrever para ficheiro](#escrever_ficheiro)
    - [write2file](#write2file)
- [Ler ficheiro](#ler_ficheiro)
    - [read2array](#read2array)
- [Testes](#testes)
    - [a) gen_huff_table](#a)
    - [b) calcular a eficiência](#b)
    - [c) codificação da mensagem](#c)
    - [d) gravar um ficheiro com a mensagem codificada](#d)
    - [e) ler do ficheiro o conjunto de bits](#e)
    - [f) descodificação da mensagem](#f)
    - [g) comparar a mensagem descodificada com a original](#g)

<a id="codificacao_huffman"></a>

# Codificação de Huffman

A codificação de Huffman é um método de compressão, desenvolvido em 1952 por  David A. Huffman, que usa as probabilidades de ocorrência dos símbolos nde um conjunto de dados a ser comprimido para determinar códigos binários de tamanho variável para cada símbolo.

Uma árvore binária completa, chamada de árvore de Huffman é construída recursivamente a partir da junção dos dois símbolos de menor probabilidade, que são então somados em símbolos auxiliares que são depois recolocados no conjunto de símbolos. O processo termina quando todos os símbolos forem unidos em símbolos auxiliares, formando uma árvore binária. A árvore é então percorrida, atribuindo-se valores binários de 1 ou 0 para cada aresta, e os códigos são gerados a partir desse percurso.

O resultado do algoritmo de Huffman pode ser visto como uma tabela de códigos de tamanho variável para codificar um símbolo. Os símbolos mais comuns são geralmente representados usando-se menos dígitos que os símbolos que aparecem com menos frequência.


Para a string **"go go gophers"**, seria gerada a seguinte árvore de Huffman e respetiva tabela:

![huff-table-example](./report/huff-table-example.PNG)

A string seria codificada como: 000 001 111 000 001 111 000 001 010 011 100 101 110. Neste caso seriam utilizados três bits por caractere (em vez de oito bits por caractere como acontece no ASCII), a string **"go go gophers"** após codificação usaria um total de 39 bits em vez de 104 bits.



# Importar bibliotecas

In [2]:
import os
from time import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json

cwd = os.getcwd() # current work diretory

In [37]:
test1 = np.frombuffer('otorrinolaringologista'.encode('utf-8'), dtype='uint8')
test2 = np.frombuffer('go go gophers'.encode('utf-8'), dtype='uint8')

<a id="io_utilities"></a>

# I/O Utilities

<a id="pad_bits"></a>

## pad_bits

Extende o número de zeros a uma sequência de bits, para permitir codificação de tamanho fixo. Os zeros são adicionados nas posições de bit mais significantes

In [4]:
def pad_bits(bits, n):
    # prefix string of bits with enough zeros to reach n digits
    if isinstance(bits, np.ndarray):
        if(n - len(bits) > 0):
            return np.pad(bits, (n - len(bits), 0))
        else:
            return bits
    else:
        return ([0] * (n - len(bits)) + bits)

<a id="to_binary_list"></a>

## to_binary_list
Converte um número inteiro na menor sequência de bits que o representa.

In [5]:
def to_binary_list(n):
    # convert integer into a list of bits
    return [n] if (n <= 1) else to_binary_list(n >> 1) + [n & 1]

    # return [int(i) for i in list('{0:0b}'.format(n))]

<a id="input_bit_reader"></a>

## InputBitReader

Para realizar compressão e descompressão com eficácia, é necessesário manipular os fluxos de dados como um fluxo de bits individuais. 

In [6]:
class InputBitReader(object): 
    def __init__(self, bit_seq): 
        self.bit_seq = bit_seq
        self.size = len(bit_seq)
        self.bits_read = 0
        self.buffer = []

    def read_bit(self):
        if self.bits_read < self.size:
            return self.read_bits(1)[0]
        else:
            return None

    def read_bits(self, n):
        self.__flush()
        if self.bits_read < self.size:
            self.buffer = pad_bits(self.bit_seq[self.bits_read:(self.bits_read + n)], n)
            self.bits_read += n
        return self.buffer
    
    def read_byte(self):
        if self.bits_read < self.size:
            byte = ''.join(list(map(str, self.read_bits(8))))
            return int(byte, 2)
        else:
            return None

    def __flush(self):
        self.buffer = []

 <a id="tabela_huffman"></a>

# Tabela de Huffman

A codificação de Huffman é um método de compressão que usa as probabilidades de ocorrência dos símbolos no conjunto de dados a ser comprimido para determinar códigos de tamanho variável para cada símbolo.



 <a id="get_symbol_frequency"></a>

## get_symbol_frequency


Lê um ficheiro, símbolo a símbolo, e retorna um dicionário com par chave-valor : símbolo-frequência, onde cada símbolo terá como respondência a sua frequência no ficheiro. 

In [25]:
# return dicionary {symbol : frequency}
def get_symbol_frequency(file):
    d = dict()
    for i in file:
        d[i] = d.setdefault(i, 0) + 1
    return d

 <a id="huff_nome"></a>

## huff_node

Classe que representa um nó de Huffman. Cada nó contêm a seguinte informação:
* o símbolo
* a frequência do símbolo
* uma ligação para a esquerda e para a direita para os seus nós filhos
* o valor de huffman atribuído quando o nó toma uma direção

In [35]:
# Huffman Node
class huff_node:
    def __init__(self, symbol, freq, left = None, right = None):
        # symbol
        self.symbol = symbol
        # frequency of symbol
        self.freq = freq
        # node left of current node
        self.left = left
        # node right of current node
        self.right = right

 <a id="create_huff_tree"></a>

## create_huff_tree

A árvore de Huffman é construída recursivamente a partir da junção dos dois símbolos de menor probabilidade, que são então somados em símbolos auxiliares e estes símbolos auxiliares recolocados no conjunto de símbolos. O processo termina quando todos os símbolos forem unidos em símbolos auxiliares, formando uma árvore binária.

1. Com o valor de cada chave única presente no dicionário, são criados nós e colocados numa lista;
2. São retirados os dois símbolos com menor frequência da lista, atribuindo-lhes o valor de 0 ou 1, e mergem-se esses dois símbolos num só, somando as suas freqûencias; 
3. O novo nó é adicionado a lista;
4. O prodecimento repete-se até enquanto o número de nós for superior a 1;
5. A função retorna o nó raiz.

In [17]:
def create_huff_tree(file):
    dictionary = get_symbol_frequency(file)
    
    tree = []
    for symb, freq in dictionary.items():
        tree.append(huff_node(symb, freq))
    
    while len(tree) > 1:
        tree = sorted(tree, key=lambda n: n.freq)
        # pop the 2 smallest nodes
        left  = tree.pop(0)
        right = tree.pop(0)
        # combine the 2 smallest nodes to create new node as their parent
        new_node = huff_node(str(left.symbol) + str(right.symbol), left.freq + right.freq, left, right)
        tree.append(new_node)
        
    return tree[0]

<a id="huff_tree2huff_dictionary"></a>

## huff_tree2huff_dictionary

A partiz de uma árvore de Huffman gera um dicionário de Huffman. A árvore é percorrida, atribuindo-se valores binários de 1 ou 0 para cada nó, e os códigos são gerados a partir desse percurso.

In [10]:
def huff_tree2huff_dictionary(node):
    d = dict()
    huff_tree2huff_dictionary_aux(node, d)
    return d

In [11]:
def huff_tree2huff_dictionary_aux(node, dictionary, value = ""):
    if(node.left):
        huff_tree2huff_dictionary_aux(node.left, dictionary, value + "0")
    if(node.right):
        huff_tree2huff_dictionary_aux(node.right, dictionary, value + "1")
    # if node is leaf
    if(not node.left and not node.right):
        dictionary[node.symbol] = value

In [18]:
huffman_tree = create_huff_tree('go go gophers')
d = huff_tree2huff_dictionary(huffman_tree)
print(d)

{'g': '00', 'o': '01', 's': '100', ' ': '101', 'p': '1100', 'h': '1101', 'e': '1110', 'r': '1111'}


<a id="huff_tree_encode"></a>

## huff_tree_encode

Além de compactar um ficheiro, é necessário também armazenar um cabeçalho no arquivo compactado que será utilizado pelo programa de descompactação. Em suma, é necessário de alguam forma, armazenar a árvore utilizada para compactar o ficheiro original. Esta necessidade deve-se ao facto de o programa de descompressão precisa também dessa mesma árvore para decodificar os dados.

Para armazenar a árvore de huffman no cabeçalho do ficheiro, recore-se a estratégia de pesquisa em árvore "post-order transversel", para assinalar cada nó visitado. Ao encontrar um nó folha, é escrito o valor 1, seguido pelo símbolo do nó folha. Ao encontrar um nó wue não seja folha, é escrito o valor um 0.

Considerando a mesma string utilizanda anteriormente como exemplo, **"go go gophers"**, a informação do cabeçalho ficaria expressa sepla seguinte codificação: **"1g1o01s1 01p1h01e1r0000"**.

In [29]:
def huff_tree_encode(node):
    bits = []
    huff_tree_encode_postorder(node, bits)
    
    huff_table = np.array([], dtype='uint8')
    for bit in bits:
        huff_table = np.append(huff_table, bit)
    huff_table = np.packbits(huff_table)
    
    return huff_table

In [30]:
def huff_tree_encode_postorder(node, bits):
    if (node.left):
        huff_tree_encode_postorder(node.left, bits)
    if (node.right):
        huff_tree_encode_postorder(node.right, bits)
    # if node is leaf
    if (not node.left and not node.right):
        bits.append(1)
        bits.append(np.unpackbits(node.symbol))
    else:
        bits.append(0)

In [19]:
# for example demonstration purposes
def huff_tree_encode_postorder_ascii_example(node, symbols):
    if (node.left):
        huff_tree_encode_postorder_ascii_example(node.left, symbols)
    if (node.right):
        huff_tree_encode_postorder_ascii_example(node.right, symbols)
    # if node is leaf
    if (not node.left and not node.right):
        symbols.append(1)
        symbols.append(node.symbol)
    else:
        symbols.append(0)

huffman_tree = create_huff_tree('go go gophers')
huff_table = []
huff_tree_encode_postorder_ascii_example(huffman_tree, huff_table)
print(''.join(list(map(str,huff_table))))

1g1o01s1 01p1h01e1r0000


<a id="huff_tree_decode"></a>

## huff_tree_decode

A construção da árvore de Huffman a partir do cabeçalho, é realizada com recurso a um stack. A informação do cabeçalho deve ser lida bit a bit. Quando se lê um bit com o valor 1, significa que se está perante um nó do tipo folha, então é lido o próximo byte e coloca-se o símbolo no stack. Quando um bit com o valor 0 é lido, se a pilha contém apenas um elemento, então toda a árvore de Huffman está construída. Caso contrário, deve haver mais de um elemento na pilha, então são retirados os dois primeiros elementos da pilha. O primeiro elemento do stack é um novo nó direito, e o segundo elemento do stack é um novo nó esquerdo. Um o nó pai é criado com os filhos nó esquerdo e direito recém-criados, e é colocado depois no stack.

In [31]:
def huff_tree_decode(huff_table):
    ibr = InputBitReader(np.unpackbits(huff_table))
    tree = []
    bits_read = 0
    
    while True:
        if (ibr.read_bit() == 1):
            byte = ibr.read_byte()
            tree.append(huff_node(byte, 0, None, None))
        else:
            # if tree contains only 1 element, then its complete
            if len(tree) == 1:
                break
            right = tree.pop()
            left  = tree.pop()
            tree.append(huff_node(0, 0, left, right))
            
    return tree[0]

<a id="gen_huff_table"></a>

## gen_huff_table

Gere todas as chamadas das funções para gerar a tabela e o dicionário de Huffman.

In [23]:
def gen_huff_table(file):
    huff_tree = create_huff_tree(file)
    huff_table = huff_tree_encode(huff_tree)
    huff_dictionary = huff_tree2huff_dictionary(huff_tree)
    return huff_table, huff_dictionary

In [34]:
# generate huff table from "go go gophers" file
encoded_table, huff_dictionary = gen_huff_table(test2)
print(huff_dictionary, '\n')

# re-construct huffman tree from huff table
decoded_huffman_tree = huff_tree_decode(encoded_table)
# get huffman dictionary from huff table
new_huff_dictionary = huff_tree2huff_dictionary(decoded_huffman_tree)
print(new_huff_dictionary, '\n')

{103: '00', 111: '01', 115: '100', 32: '101', 112: '1100', 104: '1101', 101: '1110', 114: '1111'} 

{103: '00', 111: '01', 115: '100', 32: '101', 112: '1100', 104: '1101', 101: '1110', 114: '1111'} 



 <a id="codificador_huffman"></a>

# Codificador de Huffman

<a id="encode_huff"></a>

## encode_huff

A partir de um dicionário de huffman, codifica uma mensagem. A mensagem é lida símbolo a símbolo, e o valor do símbolo é substituído pela sequência de bits presente no dicionário.

In [32]:
def encode_huff(file, huff_dictionary):
    biq_seq = ""
    for byte in file:
        biq_seq += huff_dictionary[byte]
    return list(map(int, biq_seq))

In [33]:
# generate huff table from "go go gophers" file
t, d = gen_huff_table(test2)
# encode "go go gophers" file
biq_seq = encode_huff(test2, d)
print(biq_seq)

[0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0]


 <a id="descodificador_huffman"></a>

# Descodificador de Huffman

<a id="decode_huff"></a>

## decode_huff

A partir de uma sequência de bits (mensagem codificada) e uma tabela de huffman, retorne uma mensagem descodificada. A sequência é lida bit a bit, e vão sendo colocados num buffer. Quando a sequência de bits presente no buffer corresponder a um símbolo no dicionário, este é adicionado ao output e o buffer é limpo. O processo repte-se até que todos os bits sejam lidos.

In [ ]:
def decode_huff (bit_seq, huff_table):
    huff_tree = huff_tree_decode(huff_table)
    huff_dictionary = huff_tree2huff_dictionary(huff_tree)
    inverted_huff_dictionary = dict(map(reversed, huff_dictionary.items()))
    buffer = ""
    output = []
    for bit in bit_seq:
        buffer += str(bit)
        if buffer in inverted_huff_dictionary:
            output.append(inverted_huff_dictionary[buffer])
            buffer = ""
    return bytearray(output)

In [ ]:
decoded_msg = decode_huff(biq_seq, huff_table)
print(np.unpackbits(decoded_msg))
print((str(decoded_msg, 'utf-8')))

 <a id="escrever_ficheiro"></a>

# Escrever para ficheiro

A escrita para um ficheiro é conseguida de maneira trivial, gravando primeiro um cabeçalho (tabela seguinte) com informação sobre o a codificação e seguidamente com a mensagem codificada.

Byte [index]                     | Info
:------------------------------  | :-----------------------------
0                                | tamanho da tabela de huffman
1 - huffman_table_size           | huffman 
huffman_table_size + 1           | tamanho da mensagem codificada
huffman_table_size + 2 - eof     | mensagem codificada

<a id="write2file"></a>

## write2file

In [ ]:
def write2file(huff_table, bit_seq, filename): 
    
    # calculate huffman table size
    t_size = pad_bits(to_binary_list(len(huff_table)), 8)
    table_size = np.packbits(t_size)
    
    # calculate data size
    d_size = pad_bits(to_binary_list(len(bit_seq)), 8)
    data_size = np.packbits(d_size)
    
    output = np.array([], dtype='uint8')
    
    ### header info
    # byte[0] : 
    #  size of huffman table
    output = np.append(output, table_size)
    # byte[1] - byte[table_size] : 
    #  huffman table
    output = np.append(output, huff_table)
    
    ### data info
    # byte[table_size + 1] : 
    #  size of data
    output = np.append(output, data_size)
    # byte[table_size + 1 + 1] - eof : 
    #   data
    byte_seq = np.packbits(bit_seq)
    output = np.append(output, byte_seq)
    
    np.save(f"{cwd}/data/{filename}", output)

 <a id="ler_ficheiro"></a>

# Ler ficheiro

<a id="read2array"></a>

## read2array

In [ ]:
def read2array(filename):
    # size of huffman table
    table_size = filename[0]
    # huffman table
    huff_table = filename[1:(table_size + 1)]
    # size of data
    data_size = filename[(table_size + 1)]
    # data
    byte_seq = filename[(table_size + 1 + 1):]
    # get bit array sequence from byte array
    bit_seq = np.unpackbits(byte_seq)
    # from the bit sequence we are only interested on the first data_size bits
    # so we ignore the rest
    bit_seq = bit_seq[:data_size]
    # decode bit sequence using the huff table
    return decode_huff(bit_seq, huff_table)

In [ ]:
huff_table, huff_dictionary = gen_huff_table(test2)
bit_seq = encode_huff(test2, huff_dictionary)

write2file(huff_table, bit_seq, "test")

In [ ]:
encoded_file = np.load(f"{cwd}/data/test.npy")
byte_seq = read2array(encoded_file)
print("unpack", np.unpackbits(byte_seq))
print((str(byte_seq, 'utf-8')))

 <a id="testes"></a>

# Testes

## Importar dados para teste

In [44]:
# file name
files_name = [
    'DecUniversalDH.txt',
    'DecUniversalDH.pdf',
    'HenryMancini-PinkPanther30s.mp3',
    'HenryMancini-PinkPantherC.mid',
    'LenaColor.tif',
    'LenaGray.tif',
]

# file path
files_path = []
for name in files_name:
    files_path.append(f"{cwd}/intput_data/{name}")

# files size, bytes
files_size = []
for path in files_path:
    files_size.append(os.stat(path).st_size)
    
# files, uint8    
files = []
for path in files_path:
    files.append(np.fromfile(path, dtype='uint8'))

<a id="a"></a>

## a)

In [50]:
huff_tables = []
huff_dictionaries = []

for i in range(len(files)):
    t_start = time()
    huff_table, huff_dictionary = gen_huff_table(dec_universal_IDH_txt)
    t_end = time()
    
    huff_tables.append(huff_table)
    huff_dictionaries.append(huff_dictionary)
    
    print(f"{files_name[i]}:")
    print(f"file size [bytes]: {files_size[i]}")
    print(f"time to create huff table & dictionary [seconds]: {t_end - t_start}")
    print()

DecUniversalDH.txt:
file size [bytes]: 11930
time to create huff table & dictionary [seconds]: 0.013953685760498047

DecUniversalDH.pdf:
file size [bytes]: 17524
time to create huff table & dictionary [seconds]: 0.00996851921081543

HenryMancini-PinkPanther30s.mp3:
file size [bytes]: 236925
time to create huff table & dictionary [seconds]: 0.013960838317871094

HenryMancini-PinkPantherC.mid:
file size [bytes]: 48049
time to create huff table & dictionary [seconds]: 0.014963150024414062

LenaColor.tif:
file size [bytes]: 786572
time to create huff table & dictionary [seconds]: 0.011998414993286133

LenaGray.tif:
file size [bytes]: 210122
time to create huff table & dictionary [seconds]: 0.0110015869140625



<a id="b"></a>

## b)

<a id="c"></a>

## c)

In [53]:
bit_seqs = []

for i in range(len(files)):
    t_start = time()
    bit_seq = encode_huff(files[i], huff_dictionaries[i])
    t_end = time()
    
    bit_seqs.append(bit_seq)
    
    huff_tables.append(huff_table)
    huff_dictionaries.append(huff_dictionary)
    
    print(f"{files_name[i]}:")
    print(f"file size [bytes]: {files_size[i]}")
    print(f"time to encode message [seconds]: {t_end - t_start}")
    print()

DecUniversalDH.txt:
file size [bytes]: 11930
time to encode message [seconds]: 0.01196742057800293



KeyError: 37

<a id="d"></a>

## d)

<a id="e"></a>

## e)

<a id="f"></a>

## f)

<a id="g"></a>

## g)